In [9]:
import numpy as np
import cv2

def rotate_image(img, smoothed_angle, degrees):
    rows,cols,_ = img.shape
    if smoothed_angle!=degrees:
        smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return dst, smoothed_angle

In [10]:
import os
wheel_path = "C:\\Users\\hp\\Desktop\\Self_Driving_car\\wheel-4.jpg"
wheel=cv2.imread(wheel_path)
smoothed_angle=0
smoothed_angle_actual=0
wheel.shape

(240, 240, 3)

In [11]:
import tensorflow as tf
print(tf.__version__)

2.3.1


In [12]:
model_path='C:\\Users\\hp\\Desktop\\Self_Driving_car\\m1.h5'
model = tf.keras.models.load_model(model_path,compile=False)

In [13]:
X=[]
Y=[]
lines=[]

data_path = "C:\\Users\\hp\\Desktop\\Self_Driving_car\\car_data\\data.txt"
images_path = "C:\\Users\\hp\\Desktop\\Self_Driving_car\\car_data\\images\\"


with open(data_path) as file:
    lines=file.readlines()
    
for line in lines:
    w=line.split()
    X.append(w[0])
    w1=w[1].split(',')[0]
    Y.append(float(w1))
print(Y[40])

-2.32


In [14]:
for i,x in enumerate(X):
    X[i]=images_path+X[i]
    
X[4]

'C:\\Users\\hp\\Desktop\\Self_Driving_car\\car_data\\images\\4.jpg'

In [15]:
import math

for i,y in enumerate(Y):
    Y[i]=(y*math.pi)/180
    
print(Y[89])

0.05096361415823442


In [ ]:
frame=0
error=0
cnt=0
while 1:
    img=cv2.imread(X[frame])
    cropped_img=cv2.resize(img[-150:],(200,66))/255.0
    cv2.imshow('frame',img)
    cv2.imshow('cropped_frame',cropped_img)
    
    rad=model.predict(np.array([cropped_img]))[0][0]
    
    pred_deg=(rad*180)/(math.pi)
    w,smoothed_angle=rotate_image(wheel,smoothed_angle,pred_deg)
    cv2.imshow('Prediction',w)
    
    actual_deg=(Y[frame]*180)/(math.pi)
    w2,smoothed_angle_actual=rotate_image(wheel,smoothed_angle_actual,actual_deg)
    cv2.imshow('Actual',w2)
    
    error+=(rad-Y[frame])**2
    
    cnt+=1
    MSE=error/cnt
    
    s="Prediction: "+str(rad)+" Actual: "+str(Y[frame])+" MSE: "+str(MSE)+"\n"
    s=s.encode()
    os.write(1,s)
    
    frame+=1
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()